# 01. Full Linear Regression Example
I'll walk through a simple Linear Regression example and present all performance metrics & key plots

**Key Assumptions**  
__________
- Linear relationship (Linear Regression is quite sensitive to outliers)
- Multivariate normality (check for residuals distribution, should be close to normal)
- No or little multicollinearity (VIF > 10)
- No auto-correlation (e.g.  y(x+1) is not independent from the value of y(x) in some specific cases
- Homoscedasticity (residuals are equal across the regression line)

In [1]:
# Import key liberaries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import cm  # color map 

from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

%matplotlib inline

In [2]:
from sklearn.preprocessing import PolynomialFeatures

In [3]:
# Use Boston Housing dataset from Scikit-Learn
#[Link](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_boston.html)
#create a dataframe with provided components

b_data = load_boston()
data = pd.DataFrame(data = b_data.data, columns= b_data.feature_names)
data['PRICE'] = b_data.target

# I'll pick a few features to simplify my example
data = data[['CRIM','INDUS','RM','DIS','PRICE']]

In [4]:
data.head(3)

,CRIM,INDUS,RM,DIS,PRICE
0,0.00632,2.31,6.575,4.0900,24.0
1,0.02731,7.07,6.421,4.9671,21.6
2,0.02729,7.07,7.185,4.9671,34.7


In [5]:
# Data Split
X = data.drop('PRICE', axis=1)
y = data['PRICE']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=12)

In [6]:
poly_features = PolynomialFeatures(degree=1)

In [7]:
X_train_poly = poly_features.fit_transform(X_train)
X_test_poly = poly_features.transform(X_test)

In [8]:
pd.DataFrame(X_train_poly, columns = poly_features.get_feature_names()).head(3)

,1,x0,x1,x2,x3
0,1.0,0.78570,3.97,7.014,2.1329
1,1.0,0.06263,11.93,6.593,2.4786
2,1.0,0.14103,13.92,5.790,6.3200


In [9]:
# Model Training



# Model Training - Find a way to use Stats models only! (results.rsquared)
poly_model = LinearRegression()
poly_model.fit(X_train_poly, y_train)

print('Training data r-squared:', poly_model.score(X_train_poly, y_train))
print('Test data r-squared:', poly_model.score(X_test_poly, y_test))

# # Evaluating Model Performance
# X_incl_constant = sm.add_constant(X_train)
# model=sm.OLS(y_train, X_incl_constant)
# results= model.fit()

# pd.DataFrame({'coef': results.params, 'p-value':round(results.pvalues, 5)})

Training data r-squared: 0.568086134897176
Test data r-squared: 0.598161932567424


In [10]:
poly_model.intercept_

-18.758151146486473

In [11]:
poly_model.coef_

array([ 0.        , -0.19836921, -0.35169681,  7.67096314, -0.61004115])

# Inspect model assumptions
- Check for linearity & outliers
- Check for multicolinearity and autocolinearity (if in doubt)
- Check for Multivariate normality & Homoscedasticity

In [ ]:
# Check visually for linearity & outliers
g = sns.pairplot(data,y_vars=['PRICE'], x_vars=['CRIM','INDUS','RM','DIS'], kind='reg',
                 height=4 , plot_kws={'line_kws':{'color':'red'}})
g.set(ylim=(0, 55))
plt.show()

## Testing for Multicolinearity

Variance Infilation Factor (VIF) is calculated for each feature as per this example:

$$ CRIM = \alpha _0 + \alpha _1 RM + \alpha _2 INDUS + ....+ \alpha _{4}DIS $$
$$ VIF _{CRIM} = \frac {1}{(1 - R _{CRIM} ^2)} $$


VIF > 5 there is an indication that multicollinearity may be presen  
VIF > 10 there is certainly multicollinearity among the variables

In [ ]:
# Pearson correlation coefficients(Assumption is we're using continuous data)
def get_corr(data):
    mask = np.zeros_like(data.corr())
    triangle_indices = np.triu_indices_from(mask)
    mask[triangle_indices]= True
#     plt.figure(figsize= (6,5))
    sns.heatmap(data.corr(), mask = mask, annot=True, annot_kws={"size":13})
    sns.set_style('whitegrid')
    plt.title('Pearson correlation coefficients')
    plt.xticks(fontsize = 11)
    plt.yticks(fontsize = 11)
#     plt.show()

    
# Calculate the Variance Infilation Factor for all parameters
def get_vif(X_incl_constant):
    vif = []
    for i in range(1, X_incl_constant.shape[1]):
        vif.append(variance_inflation_factor(exog=X_incl_constant.values, exog_idx=i))
    vif_data = pd.DataFrame({'coef_name': X_incl_constant.columns[1:X_incl_constant.shape[1]], 'VIF': np.around(vif, 2)})
    
    plt.bar(vif_data['coef_name'], height = vif_data['VIF'] )
    plt.hlines(10, -0.5, X_incl_constant.shape[1]-1.5,  linestyles='--', colors='red', label='VIF threshold' )
    plt.title('Variance Infilation Factor plot')
    plt.ylabel('VIF')
    plt.legend()
    for i, v in enumerate(vif_data['VIF']):
        plt.text(i-0.1  , v + 0.5  , str(v), color='black', fontweight='bold')
#     plt.show()

In [ ]:
plt.figure(figsize=(14,5))
plt.subplot(121)
get_corr(data)
plt.subplot(122)
get_vif(X_incl_constant)

## Residuals analysis

In [ ]:
# Residuals

# residuals vs predicted values
plt.figure(figsize=(15,6))
plt.subplot(121)
plt.scatter(x= results.fittedvalues, y= results.resid, c='blue', alpha=0.6)
plt.axhline(0, 0, 1, color='black',ls = '--')
plt.xlabel('Fitted Values', fontsize=None)
plt.ylabel('Residuals', fontsize=None)
plt.title('Residuals vs Fitted values')

plt.subplot(122)
# Distribution of Residuals (log prices) - checking for normality
resid_mean = round(results.resid.mean(), 3)
resid_skew= round(results.resid.skew(), 3)
sns.distplot(results.resid, color='blue')
plt.title(f'Distribution of Residuals: residuals Skew({resid_skew}) Mean({resid_mean})')
plt.show()


## Results Summary
- Adj. R-squared: good measure to adjust for R2 increases due to added features
- Baisian Information Criterion (BIC): to show the model complexity - compare the BIC between mutiple models and use the lower value

In [ ]:
print(results.summary())